# Import

In [116]:
data_path = '/content/drive/MyDrive/ColabNotebooks/data'

In [117]:
from google.colab import drive
import zipfile
from pathlib import Path

import os
# from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from scipy.fftpack import fft, rfft
from sklearn import preprocessing
# from scipy.fft import fft

# Монтуємо GoogleDrive та готуємо архів

In [ ]:
# !mkdir -p /content/gdrive
drive.mount('/content/gdrive') #, force_remount=True)

Mounted at /content/gdrive


In [ ]:
# zip_path = '/content/drive/MyDrive/ColabNotebooks/homework.zip'
# extract_path = '/content/drive/MyDrive/ColabNotebooks/'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# Open data

In [136]:
# перевірка правильності розархівування
def count_files_in_folder(folder_path):
    file_count = 0
    for root, dirs, files in os.walk(folder_path):
        file_count += len(files)
    return file_count

zip_path = '/content/drive/MyDrive/ColabNotebooks/homework.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_file_list = zip_ref.namelist()

zip_file_count = len(zip_file_list)
print(f"Кількість файлів у зіп-архіві: {zip_file_count}")

extracted_folder_path = '/content/drive/MyDrive/ColabNotebooks/data'
extracted_folder_count = count_files_in_folder(extracted_folder_path)
print(f"Кількість файлів у розархівованій теці: {extracted_folder_count}")

if zip_file_count == extracted_folder_count:
    print("Кількість файлів - однакова.")
else:
    print("Кількість файлів - різна.")

Кількість файлів у зіп-архіві: 6466
Кількість файлів у розархівованій теці: 6462
Кількість файлів - різна.


In [137]:
# drive.flush_and_unmount()
# drive.mount('/content/gdrive', force_remount=True)

In [138]:
data_path = Path(data_path)
try:
    if data_path.exists() and data_path.is_dir():
        activities = [entry.name for entry in data_path.iterdir() if entry.is_dir()]
        print("Список директорій:", activities)
    else:
        print(f"Директорія {data_path} не існує або не є директорією.")
except Exception as e:
    print(f"Виникла помилка: {e}")


Список директорій: ['idle', 'running', 'stairs', 'walking']


In [139]:
# from google.colab import drive
# drive.mount('/content/drive')

In [140]:
# Перевірка підключення і синхронізації Гугл диску
print("Список файлів та директорій у шляху:")
for entry in data_path.iterdir():
    print(entry)

Список файлів та директорій у шляху:
/content/drive/MyDrive/ColabNotebooks/data/idle
/content/drive/MyDrive/ColabNotebooks/data/running
/content/drive/MyDrive/ColabNotebooks/data/stairs
/content/drive/MyDrive/ColabNotebooks/data/walking


In [141]:
# activities = Path(data_path)
# activities

In [142]:
# ['idle', 'walking', 'sairs', 'running']

In [143]:
activities = [entry.name for entry in Path(data_path).
              iterdir() if entry.is_dir()]

activities

['idle', 'running', 'stairs', 'walking']

In [144]:
for act in activities:
    path = join(data_path, act)
    frames = Path(path)
    file_count = len(list(frames.iterdir()))
    print(act, ': ', file_count)

idle :  1039
running :  3408
stairs :  165
walking :  1850


In [145]:
activity_folder = "walking"

frames = Path(join(data_path, activity_folder))
csv_files = [entry for entry in frames.
             iterdir() if entry.is_file() and entry.suffix == '.csv']
file_to_read = csv_files[5]
frame = pd.read_csv(file_to_read)
frame.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-4.462792,11.176133,14.006080
1,1.393425,-13.579912,-4.328717
2,-1.441309,-8.815451,-2.834735
3,-6.349423,-15.978902,5.439626
4,-4.180276,1.776498,1.345541


# Data preparating

In [131]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from scipy.fftpack import fft, rfft
# from sklearn import preprocessing
# from scipy.fft import fft

## Feature calculation functions

In [146]:

features = ['skew_X','skew_Y','skew_Z',
            'kurtosis_X','kurtosis_Y','kurtosis_Z',
            'max_X','max_Y','max_Z',
            'min_X','min_Y','min_Z',
            'mean_X','mean_Y','mean_Z',
            'std_X','std_Y','std_Z',
            'variance_X','variance_Y','variance_Z',
            'median_X', 'median_Y', 'median_Z',
            'index_max_X','index_max_Y','index_max_Z',
            'index_min_X','index_min_Y','index_min_Z',
            'correlation_X_Y',
            'correlation_X_Z',
            'correlation_Y_Z',
            'mae_X','mae_Y','mae_Z',
            'rmse_X','rmse_Y','rmse_Z']
len(features)

39

Create function that will be calculated all statistic features

In [147]:
def get_stat_features(frame):
    features = np.array([])
    # features = []
    # features = np.array()

    features = np.append(features, frame.skew(axis=0).values)
    features = np.append(features, frame.kurt(axis=0).values)
    features = np.append(features, frame.max(axis=0).values)
    features = np.append(features, frame.min(axis=0).values)
    features = np.append(features, frame.mean(axis=0).values)
    features = np.append(features, frame.std(axis=0).values)
    features = np.append(features, frame.var(axis=0).values)
    features = np.append(features, frame.median(axis=0).values)
    features = np.append(features, frame.idxmax(axis=0).values)
    features = np.append(features, frame.idxmin(axis=0).values)

    correlations = frame.corr()
    corr = np.array([
        correlations.loc['accelerometer_X', 'accelerometer_Y'],
        correlations.loc['accelerometer_X', 'accelerometer_Z'],
        correlations.loc['accelerometer_Y', 'accelerometer_Z']
    ])
    features = np.append(features, corr)
    # features = np.concatenate((features, corr), axis=0)

    frame['mean_X'] = frame.mean(axis=0)['accelerometer_X']
    frame['mean_Y'] = frame.mean(axis=0)['accelerometer_Y']
    frame['mean_Z'] = frame.mean(axis=0)['accelerometer_Z']

    mae_X = np.sqrt(mean_absolute_error(frame['accelerometer_X'], frame['mean_X']))
    mae_Y = np.sqrt(mean_absolute_error(frame['accelerometer_Y'], frame['mean_Y']))
    mae_Z = np.sqrt(mean_absolute_error(frame['accelerometer_Z'], frame['mean_Z']))

    rmse_x = np.sqrt(mean_squared_error(frame['accelerometer_X'], frame['mean_X']))
    rmse_y = np.sqrt(mean_squared_error(frame['accelerometer_Y'], frame['mean_Y']))
    rmse_z = np.sqrt(mean_squared_error(frame['accelerometer_Z'], frame['mean_Z']))

    metrics = np.array([mae_X, mae_Y, mae_Z, rmse_x, rmse_y, rmse_z])
    features = np.append(features, metrics)

    return features

features_result = get_stat_features(frame)

In [148]:
len(get_stat_features(frame))

69

Create function that will be calculate all Fast Fourier Transform coefficients

In [149]:
def get_fft_features(frame):
  fft_X = np.array(preprocessing.normalize([rfft(frame['accelerometer_X'].values)])[0])
  fft_Y = np.array(preprocessing.normalize([rfft(frame['accelerometer_Y'].values)])[0])
  fft_Z = np.array(preprocessing.normalize([rfft(frame['accelerometer_Z'].values)])[0])

  features = np.concatenate((fft_X, fft_Y),axis=0)
  features = np.concatenate((features, fft_Z),axis=0)

  return features


In [150]:
len(get_fft_features(frame))

90

## Discover calculation of features

Перевірка розрахунків (чернетка)

In [151]:
a = np.array([2])

In [152]:
frame.skew(axis=0).values
a = np.concatenate((a, frame.skew(axis=0).values),axis=0)

a

array([ 2.        , -0.37636669,  0.88429766,  1.4832776 ,  0.        ,
        0.        ,  0.        ])

In [153]:
frame.kurt(axis=0)

accelerometer_X    0.830809
accelerometer_Y    0.913417
accelerometer_Z    4.196724
mean_X             0.000000
mean_Y             0.000000
mean_Z             0.000000
dtype: float64

In [154]:
frame.max(axis=0)

accelerometer_X     9.370906
accelerometer_Y    12.124237
accelerometer_Z    23.769634
mean_X             -0.124179
mean_Y             -8.126878
mean_Z              0.016600
dtype: float64

In [155]:
frame.min(axis=0)

accelerometer_X   -14.805743
accelerometer_Y   -21.222204
accelerometer_Z   -10.888829
mean_X             -0.124179
mean_Y             -8.126878
mean_Z              0.016600
dtype: float64

In [156]:
frame.mean(axis=0)

accelerometer_X   -0.124179
accelerometer_Y   -8.126878
accelerometer_Z    0.016600
mean_X            -0.124179
mean_Y            -8.126878
mean_Z             0.016600
dtype: float64

In [157]:
frame.std(axis=0)

accelerometer_X    5.319076e+00
accelerometer_Y    7.919310e+00
accelerometer_Z    6.865318e+00
mean_X             4.234510e-17
mean_Y             0.000000e+00
mean_Z             0.000000e+00
dtype: float64

In [158]:
frame.var(axis=0)

accelerometer_X    2.829257e+01
accelerometer_Y    6.271547e+01
accelerometer_Z    4.713260e+01
mean_X             1.793107e-33
mean_Y             0.000000e+00
mean_Z             0.000000e+00
dtype: float64

In [159]:
frame.idxmax(axis=0)

accelerometer_X    25
accelerometer_Y    17
accelerometer_Z    19
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

# Prepare one class (calculation features)

Підготовка фічів кожного з класів

In [127]:
def class_data_prepare(class_name, class_number):
  path = join(data_path, class_name)

  X=[]

  for item in os.listdir(path):
    frame = pd.read_csv(join(path, item))
    frame = frame.values
    frame = frame.flatten()
    X.append(frame)

  y = [class_number]*len(X)

  X = np.array(X)
  y = np.array(y)

  return X, y


In [128]:
def class_data_stat_fft_prepare(class_name, class_number):
  path = join(data_path, class_name)

  X=[]

  for item in os.listdir(path):
    frame = pd.read_csv(join(path, item))
    features_stat = get_stat_features(frame)
    features_fft = get_fft_features(frame)
    features = np.concatenate((features_stat, features_fft), axis=0)
    X.append(features)

  y = [class_number]*len(X)

  X = np.array(X)
  y = np.array(y)

  return X, y

In [129]:
def class_data_fft_prepare(class_name, class_number):
  path = join(data_path, class_name)

  X=[]

  for item in os.listdir(path):
    frame = pd.read_csv(join(path, item))
    features = get_fft_features(frame)
    X.append(features)

  y = [class_number]*len(X)

  X = np.array(X)
  y = np.array(y)

  return X, y

In [130]:
def class_data_stat_prepare(class_name, class_number):
  path = join(data_path, class_name)

  X=[]

  for item in os.listdir(path):
    frame = pd.read_csv(join(path, item))
    features = get_stat_features(frame)
    X.append(features)

  y = [class_number]*len(X)

  X = np.array(X)
  y = np.array(y)

  return X, y

# Create DataSet

In [160]:
def create_dataset(class_prepare):
  X_idle, y_idle = class_prepare('idle', 0)
  X_walking, y_walking = class_prepare('walking', 1)
  X_stairs, y_stairs = class_prepare('stairs', 2)
  X_running, y_running = class_prepare('running', 3)

  X = np.concatenate((X_idle, X_walking), axis=0)
  X = np.concatenate((X, X_stairs), axis=0)
  X = np.concatenate((X, X_running), axis=0)

  Y = np.concatenate((y_idle, y_walking), axis=0)
  Y = np.concatenate((Y, y_stairs), axis=0)
  Y = np.concatenate((Y, y_running), axis=0)

  return X, Y

In [161]:
X_fft, Y_fft = create_dataset(class_data_fft_prepare)

In [162]:
with open('fft_data_X.pickle', 'wb') as f:
  pickle.dump(X_fft, f)

with open('fft_data_Y.pickle', 'wb') as f:
  pickle.dump(Y_fft, f)

In [163]:
X_fft_stat, Y_fft_stat = create_dataset(class_data_stat_fft_prepare)

In [164]:
X_fft_df = pd.DataFrame(X_fft, columns=[f'feature_{i}' for i in range(X_fft.shape[1])])
print(X_fft_df.head())

   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.998698   0.005624  -0.004901   0.000260   0.000459  -0.005633   
1   0.998766   0.008917  -0.001384   0.001151   0.001963  -0.010603   
2   0.605491  -0.230729  -0.057244  -0.215533  -0.111942  -0.181303   
3   0.541231  -0.023571  -0.202445   0.199188  -0.017526   0.018689   
4   0.550287  -0.217141  -0.140652  -0.099684  -0.228097   0.053383   

   feature_6  feature_7  feature_8  feature_9  ...  feature_80  feature_81  \
0   0.009981  -0.011096   0.004091  -0.019137  ...    0.000184    0.000129   
1  -0.000395  -0.003110  -0.009482   0.021267  ...    0.000425    0.000391   
2  -0.168153  -0.124956  -0.194938  -0.085971  ...   -0.003072   -0.004459   
3   0.238683  -0.247399   0.082455  -0.167776  ...   -0.007313    0.009341   
4  -0.222975   0.147210  -0.116246   0.148516  ...    0.008797   -0.008260   

   feature_82  feature_83  feature_84  feature_85  feature_86  feature_87  \
0    0.000481    0.000510  

# Old Data preparations

## Idle

In [165]:
import pandas as pd

In [166]:
idle_path = '/content/drive/MyDrive/ColabNotebooks/data/idle'
idle =[]
for item in os.listdir(idle_path):
  frame = pd.read_csv(join(idle_path, item))
  example = frame.values
  example = example.flatten()
  idle.append(example)

idle

[array([ 3.639190e-01,  2.394200e-02,  9.787497e+00,  3.926490e-01,
        -2.873000e-02,  9.725247e+00,  4.261680e-01, -4.309600e-02,
         9.768343e+00,  4.261680e-01,  4.788000e-03,  9.739613e+00,
         3.782840e-01, -1.436500e-02,  9.744401e+00,  4.596870e-01,
        -4.309600e-02,  9.763555e+00,  3.639190e-01,  1.436500e-02,
         9.710882e+00,  4.213800e-01, -2.873000e-02,  9.758766e+00,
         4.357450e-01, -6.224900e-02,  9.773131e+00,  3.974370e-01,
         9.577000e-03,  9.734824e+00,  4.022260e-01, -2.873000e-02,
         9.773131e+00,  3.926490e-01, -9.577000e-03,  9.749189e+00,
         4.022260e-01, -1.436500e-02,  9.749189e+00,  3.782840e-01,
        -4.309600e-02,  9.758766e+00,  4.165910e-01,  9.577000e-03,
         9.749189e+00,  4.165910e-01, -9.577000e-03,  9.739613e+00,
         4.022260e-01, -9.577000e-03,  9.749189e+00,  3.782840e-01,
        -1.915400e-02,  9.749189e+00,  4.070140e-01, -3.351900e-02,
         9.758766e+00,  3.878610e-01,  2.873000e

In [167]:
y_idle = [0]*len(idle)

In [168]:
len(idle)

1039

In [169]:
idle_folder = "idle"

idle_frames = Path(join(data_path, activity_folder))
csv_files = [entry for entry in idle_frames.
             iterdir() if entry.is_file() and entry.suffix == '.csv']
file_to_read = csv_files[5]
idle_frame = pd.read_csv(file_to_read)
idle_frame.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-4.462792,11.176133,14.006080
1,1.393425,-13.579912,-4.328717
2,-1.441309,-8.815451,-2.834735
3,-6.349423,-15.978902,5.439626
4,-4.180276,1.776498,1.345541


## Stairs

In [170]:
stairs_path = '/content/drive/MyDrive/ColabNotebooks/data/stairs'
stairs =[]
for item in os.listdir(stairs_path):
  frame = pd.read_csv(join(stairs_path, item))
  example = frame.values
  example = example.flatten()
  stairs.append(example)

stairs

[array([  3.529053,  -7.43639 ,   1.905785,   0.986411, -17.3005  ,
         -0.383072,  -4.582502,  -7.862558,   9.792285,   1.910573,
         -5.985504,  -1.599327,   4.458004,  -6.914454,  -5.401319,
          2.475605,  -8.705317,  -0.373495,  -1.120486,  -5.803545,
          2.044648,  -2.39899 ,  -8.451532,   4.055778,   5.64074 ,
         -8.566454,   1.029507,  -0.828394,  -0.483629,  10.582372,
          4.491523,  -7.393295,  -1.58975 ,   4.97994 , -10.127473,
        -14.422671,   1.106121,  -7.115567,   2.240973,   1.355118,
         -8.66701 ,   1.479617,   2.149993,  -7.474697,   5.93762 ,
          8.288726, -10.783484,  -3.165135,  -0.991199, -19.416977,
          4.074931,   7.062895,  -8.063671,  -0.972046,   0.766145,
         -4.788403,  -0.981623,   0.02873 ,  -6.545747,  -0.646434,
          6.952762, -11.247959,   0.344765,   6.895301, -13.982138,
         -2.504335,  -1.417367, -19.349937,   0.177171,   0.320823,
         -3.715801,  -9.653421,  -4.649539, -15.

In [171]:
stairs = [0]*len(stairs)

In [172]:
len(stairs)

165

# Load DataSet

In [173]:
# import pickle

def open_data_set(flag):
  name = flag + '_data_'

  with open(name + 'X.pickle', 'rb') as f:
    X = pickle.load(f)

  with open(name + 'Y.pickle', 'rb') as f:
    Y = pickle.load(f)

  return X,Y

You can load DataSet from corresponding pickle apply this function uses flags

In [174]:
# 'fft'
# 'stat'
# 'fft_stat'

fft_stat = pd.DataFrame()

In [175]:
fft_stat.head()

""


for Fast Fourier Transform features, Statistical features and ast flag for

In [176]:
X, Y = open_data_set('fft')

Ділимо дані на тренувальний та тестовий

In [177]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)


In [178]:
np.array_equal(X, X_fft_stat)

False

# SVM Classifier

In [179]:
# from sklearn import svm

classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='rbf',
                     gamma=0.005,
                     probability=True)


In [180]:
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.005, probability=True)

In [181]:
y_test

array([3, 0, 1, ..., 1, 0, 0])

In [182]:
y_test_pred = classifier.decision_function(X_test)

y_test_pred

array([[-0.02327186, -0.16095503, -2.18017834,  1.03701932, -1.21905404,
        -1.18847554],
       [ 1.08895065,  1.02206684,  0.90564793,  1.19976498, -1.5386117 ,
        -1.64942151],
       [-1.7441748 , -0.69243088, -0.59144355,  1.0336235 ,  1.24479627,
         1.00706738],
       ...,
       [-0.77357027, -0.19934145, -0.5731772 ,  1.02553639,  1.0407861 ,
         0.80375181],
       [ 1.24812307,  1.06112401,  0.55589404,  1.20313016, -1.5923026 ,
        -1.70155618],
       [ 1.0029552 ,  1.00291658,  1.06218302,  1.08171418,  0.57430303,
         0.27592803]])

In [183]:
# y_test_proba = classifier.predict_function(X_test)
y_test_proba = classifier.predict_proba(X_test)


y_test_proba

array([[2.35385290e-07, 2.73451954e-07, 1.72021699e-07, 9.99999319e-01],
       [9.99339661e-01, 5.81953654e-05, 5.91566702e-04, 1.05769879e-05],
       [7.87536937e-05, 9.86659326e-01, 1.30846189e-02, 1.77301862e-04],
       ...,
       [2.60828275e-03, 9.73957841e-01, 2.26601188e-02, 7.73757192e-04],
       [9.99237008e-01, 1.77594165e-05, 4.45144120e-04, 3.00088638e-04],
       [9.98710328e-01, 2.22135988e-04, 9.93329316e-04, 7.42071692e-05]])

In [184]:
y_test_proba = np.argmax(y_test_proba, axis=1)

y_test_proba

array([3, 0, 1, ..., 1, 0, 0])

In [185]:
count_idle = 0
count_walking = 0
count_stairs = 0
count_running = 0

for i in y_test_proba:
  if i == 0 :
    count_idle += 1
  elif i ==1 :
    count_walking += 1
  elif i == 2 :
    count_stairs += 1
  else:
    count_running += 1

In [186]:
print(f'{count_idle}')
print(f'{count_walking}')
print(f'{count_stairs}')
print(f'{count_running}')

307
584
32
1016


In [187]:
# from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_test_proba)

accuracy

0.9742135121196493

In [188]:
print(f"{classifier.classes_=}")
print(f"{classifier.score(X_test, y_test)=}")

classifier.classes_=array([0, 1, 2, 3])
classifier.score(X_test, y_test)=0.9654461062403301


In [189]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     0.9902    0.9806    0.9854       310
     running     0.9469    0.9822    0.9643       563
      stairs     0.6875    0.4151    0.5176        53
     walking     0.9941    0.9970    0.9956      1013

    accuracy                         0.9742      1939
   macro avg     0.9047    0.8438    0.8657      1939
weighted avg     0.9714    0.9742    0.9718      1939



# SVM Classifier on stat features

In [190]:
# with open('fft_data_X.pickle', 'rb') as f:
#     X_stat_train = pickle.load(f)

# with open('fft_data_Y.pickle', 'rb') as f:
#     Y_stat_train = pickle.load(f)

In [191]:
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import StandardScaler

# X_train, X_test, y_train, y_test = train_test_split(X_stat_train, Y_stat_train, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# classifier = SVC()
# classifier.fit(X_train_scaled, y_train)

# y_pred = classifier.predict(X_test_scaled)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.4f}")

In [192]:
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import f1_score, accuracy_score
# from sklearn import svm

In [193]:
classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='rbf',
                     gamma=0.01,
                     probability=True)


In [194]:
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.01, probability=True)

In [195]:
scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.973026431459884

In [196]:
y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9804022692109334

In [197]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9503    0.9840    0.9668       563
      stairs     0.7273    0.4528    0.5581        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9804      1939
   macro avg     0.9194    0.8592    0.8812      1939
weighted avg     0.9781    0.9804    0.9783      1939



# With scale gamma

In [198]:
classifier = svm.SVC(decision_function_shape='ovo', probability=True)

In [199]:
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', probability=True)

In [200]:
scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9814291156303112

In [201]:
y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9819494584837545

In [202]:
# make_report(y_test_proba, y_test)
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9536    0.9858    0.9694       563
      stairs     0.7647    0.4906    0.5977        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9819      1939
   macro avg     0.9296    0.8691    0.8918      1939
weighted avg     0.9801    0.9819    0.9801      1939



# With linear kernel

In [203]:
classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='linear',
                     probability=True)

In [204]:
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [205]:
scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9765662544687336

In [206]:
y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9798865394533265

In [207]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9471    0.9858    0.9661       563
      stairs     0.7333    0.4151    0.5301        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9201    0.8502    0.8740      1939
weighted avg     0.9774    0.9799    0.9773      1939



In [208]:
classifier.coef_.mean(axis=0)

array([-3.37073378e-01,  1.61324781e-02, -6.58426151e-02,  3.01274296e-02,
       -7.10610153e-02, -2.26600357e-02, -9.89794772e-02, -5.75134588e-02,
       -2.63172585e-02,  5.43568414e-02,  4.38994798e-02, -1.68955383e-01,
        2.91200476e-02,  1.05142773e-02,  2.15460130e-03, -4.04320465e-02,
       -1.35605056e-02,  6.98847663e-02, -1.43168718e-02, -2.98254981e-02,
       -8.38528591e-02, -5.74494370e-02,  4.23233804e-02,  8.03253918e-02,
       -1.50468967e-02,  5.38575501e-02, -1.56015934e-02,  5.20720809e-02,
       -2.00878628e-02, -1.18472566e-01, -4.23946760e-02, -1.51194784e-01,
       -2.24779455e-01, -5.29545062e-02, -8.13935560e-02, -2.21152904e-02,
       -6.35934325e-02,  4.08373985e-03, -6.36540213e-02,  9.20019877e-02,
       -4.51476590e-03,  3.85700751e-02, -3.20819291e-02,  2.40603402e-02,
        5.29062435e-03, -1.10529837e-01,  4.10924565e-02, -7.32203647e-02,
       -7.66070009e-03,  3.47637780e-02,  4.79056255e-02,  1.01101656e-01,
       -1.49532121e-01, -

# SVM OVR with Linear Kernel

change one on rest

In [209]:
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import f1_score, accuracy_score
# from sklearn import svm

In [210]:
cls_ovr = svm.SVC(kernel='linear', probability=True)

In [211]:
scores = cross_val_score(cls_ovr, X_train, y_train, cv=10)
scores.mean()

0.9765662544687336

In [212]:
cls_ovr.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [213]:
y_test_proba = cls_ovr.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9783393501805054

In [214]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9408    0.9876    0.9636       563
      stairs     0.7200    0.3396    0.4615        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9783      1939
   macro avg     0.9152    0.8318    0.8563      1939
weighted avg     0.9752    0.9783    0.9747      1939



In [215]:
cls_ovr.coef_.shape

(6, 90)

In [216]:
feature_importances = np.abs(cls_ovr.coef_).mean(axis=0)

In [217]:
print('Feature importances scaled (*1000)')
for i, f in zip(feature_importances, features):
  print(f, ' --- ', i*1000)

Feature importances scaled (*1000)
skew_X  ---  338.00642743313244
skew_Y  ---  83.71769641476556
skew_Z  ---  89.75611611150057
kurtosis_X  ---  32.51664733672951
kurtosis_Y  ---  71.06101525776242
kurtosis_Z  ---  46.699065988789506
max_X  ---  108.51892180808231
max_Y  ---  78.07797559101714
max_Z  ---  30.61130647792143
min_X  ---  62.26638436726994
min_Y  ---  63.286966296680404
min_Z  ---  168.95538308499638
mean_X  ---  49.576746295596976
mean_Y  ---  38.17795084581197
mean_Z  ---  17.779011990452787
std_X  ---  49.76369899577205
std_Y  ---  19.710588331912803
std_Z  ---  90.21509177560107
variance_X  ---  43.26267179764077
variance_Y  ---  34.22717687826316
variance_Z  ---  83.8528591495221
median_X  ---  66.82423087114806
median_Y  ---  81.94031590379578
median_Z  ---  108.84991192174044
index_max_X  ---  20.5365284132642
index_max_Y  ---  60.13906943615049
index_max_Z  ---  67.30168719928596
index_min_X  ---  63.94834493332889
index_min_Y  ---  110.8706737218419
index_min_Z  

# SVM on FFT Data

In [221]:
classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='rbf',
                     gamma=0.05,
                     probability=True)

scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9732481587841137

In [222]:

classifier.fit(X_train, y_train)

y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9804022692109334

In [223]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9549    0.9787    0.9667       563
      stairs     0.6923    0.5094    0.5870        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9804      1939
   macro avg     0.9118    0.8720    0.8884      1939
weighted avg     0.9785    0.9804    0.9790      1939



# SVM with scaled gamma

In [224]:

classifier = svm.SVC(decision_function_shape='ovo',
                     probability=True)

scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9814291156303112

In [225]:
classifier.fit(X_train, y_train)

y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9814337287261475

In [226]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9535    0.9840    0.9685       563
      stairs     0.7429    0.4906    0.5909        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9814      1939
   macro avg     0.9241    0.8686    0.8899      1939
weighted avg     0.9795    0.9814    0.9797      1939



# SVM with Linear Kernel

In [227]:
classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='linear',
                     probability=True)

scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9765662544687336

In [228]:
classifier.fit(X_train, y_train)

y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9798865394533265

In [229]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9456    0.9876    0.9661       563
      stairs     0.7500    0.3962    0.5185        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9239    0.8459    0.8712      1939
weighted avg     0.9774    0.9799    0.9770      1939



# SVM on Stat + FFT Data

In [230]:
classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='rbf',
                     gamma=0.05,
                     probability=True)

scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9732481587841137

In [231]:
classifier.fit(X_train, y_train)

y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9809179989685405

In [232]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9534    0.9822    0.9676       563
      stairs     0.7222    0.4906    0.5843        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9809      1939
   macro avg     0.9189    0.8682    0.8880      1939
weighted avg     0.9789    0.9809    0.9792      1939



# SVM on Stat + FFT Data - **With scale gamma**

In [233]:
classifier = svm.SVC(decision_function_shape='ovo',
                     probability=True)

scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9814291156303112

In [234]:
classifier.fit(X_train, y_train)

y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9814337287261475

In [235]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9535    0.9840    0.9685       563
      stairs     0.7429    0.4906    0.5909        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9814      1939
   macro avg     0.9241    0.8686    0.8899      1939
weighted avg     0.9795    0.9814    0.9797      1939



# SVM on Stat + FFT Data - **Linear Kernel**

In [236]:
classifier = svm.SVC(decision_function_shape='ovo',
                     kernel='linear',
                     probability=True)

scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9765662544687336

In [237]:
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [238]:

y_test_proba = classifier.predict_proba(X_test)
y_test_proba = np.argmax(y_test_proba, axis=1)
accuracy = accuracy_score(y_test, y_test_proba)
accuracy

0.9798865394533265

In [239]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9471    0.9858    0.9661       563
      stairs     0.7333    0.4151    0.5301        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9201    0.8502    0.8740      1939
weighted avg     0.9774    0.9799    0.9773      1939



In [240]:
classifier.coef_.mean(axis=0)

array([-3.37073378e-01,  1.61324781e-02, -6.58426151e-02,  3.01274296e-02,
       -7.10610153e-02, -2.26600357e-02, -9.89794772e-02, -5.75134588e-02,
       -2.63172585e-02,  5.43568414e-02,  4.38994798e-02, -1.68955383e-01,
        2.91200476e-02,  1.05142773e-02,  2.15460130e-03, -4.04320465e-02,
       -1.35605056e-02,  6.98847663e-02, -1.43168718e-02, -2.98254981e-02,
       -8.38528591e-02, -5.74494370e-02,  4.23233804e-02,  8.03253918e-02,
       -1.50468967e-02,  5.38575501e-02, -1.56015934e-02,  5.20720809e-02,
       -2.00878628e-02, -1.18472566e-01, -4.23946760e-02, -1.51194784e-01,
       -2.24779455e-01, -5.29545062e-02, -8.13935560e-02, -2.21152904e-02,
       -6.35934325e-02,  4.08373985e-03, -6.36540213e-02,  9.20019877e-02,
       -4.51476590e-03,  3.85700751e-02, -3.20819291e-02,  2.40603402e-02,
        5.29062435e-03, -1.10529837e-01,  4.10924565e-02, -7.32203647e-02,
       -7.66070009e-03,  3.47637780e-02,  4.79056255e-02,  1.01101656e-01,
       -1.49532121e-01, -

# Random Forest

In [241]:
# from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=15, max_depth=None)

In [242]:
clf = clf.fit(X_train, y_train)

In [243]:
scores = cross_val_score(clf, X_train, y_train, cv=10)

In [244]:
scores.mean()

0.9778912461661685

In [245]:
y_pred = clf.predict(X_test)

In [246]:
y_test_pred = classifier.predict(X_test)

print("y_test:", y_test)
print("y_test_pred:", y_test_pred)

y_test: [3 0 1 ... 1 0 0]
y_test_pred: [3 0 1 ... 1 0 0]


In [247]:
accuracy = accuracy_score(y_test, y_test_pred)
accuracy

0.9798865394533265

In [248]:
from sklearn.metrics import f1_score
f1score = f1_score(y_test, y_pred, average='macro')
f1score

0.813178935759581

In [249]:
count_idle = 0
count_walking = 0
count_stairs = 0
count_running = 0

for i in y_pred:
  if i == 0 :
    count_idle += 1
  elif i == 1 :
    count_walking += 1
  elif i == 2 :
    count_stairs += 1
  elif i == 3 :
    count_running += 1

print ('idle', count_idle)
print ('walking', count_walking)
print ('stairs', count_stairs)
print ('running', count_running)

idle 310
walking 607
stairs 9
running 1013


In [250]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9471    0.9858    0.9661       563
      stairs     0.7333    0.4151    0.5301        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9201    0.8502    0.8740      1939
weighted avg     0.9774    0.9799    0.9773      1939



# Random Forest Stat Data

In [251]:
rf_stat = RandomForestClassifier(n_estimators=50, max_depth=None)

In [252]:
scores = cross_val_score(rf_stat, X_train, y_train, cv=10)
scores.mean()

0.9781129734903985

In [253]:
rf_stat = rf_stat.fit(X_train, y_train)
y_pred = rf_stat.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9793708096957194

In [254]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9471    0.9858    0.9661       563
      stairs     0.7333    0.4151    0.5301        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9201    0.8502    0.8740      1939
weighted avg     0.9774    0.9799    0.9773      1939



In [255]:
feature_importances = rf_stat.feature_importances_

In [256]:
print('Feature importances scaled (*1000)')
for i, f in zip(feature_importances, features):
  print(f, ' --- ', i*1000)

Feature importances scaled (*1000)
skew_X  ---  144.56704063598406
skew_Y  ---  2.10591227474728
skew_Z  ---  3.29105624743027
kurtosis_X  ---  1.27207568828454
kurtosis_Y  ---  1.9104067829526523
kurtosis_Z  ---  2.28799825035701
max_X  ---  2.32148163673584
max_Y  ---  2.143400170248327
max_Z  ---  2.243397680761068
min_X  ---  1.9628030130246799
min_Y  ---  2.095309038871115
min_Z  ---  1.3850411837600198
mean_X  ---  1.291574430225595
mean_Y  ---  1.0453548058242779
mean_Z  ---  1.5645420017049738
std_X  ---  1.8115871369975256
std_Y  ---  2.4917236433806846
std_Z  ---  1.1085087047710747
variance_X  ---  1.1608599686979248
variance_Y  ---  1.2317407525712019
variance_Z  ---  1.5224147877352574
median_X  ---  2.115099917836644
median_Y  ---  2.0016209100946942
median_Z  ---  1.2608791886587725
index_max_X  ---  1.7800952768613618
index_max_Y  ---  0.9345428744307703
index_max_Z  ---  1.187430256664422
index_min_X  ---  1.7604500977423239
index_min_Y  ---  1.9452969526157076
index_m

# Random Forest FFT Data

In [257]:
rf_fft = RandomForestClassifier(n_estimators=15, max_depth=None)

scores = cross_val_score(rf_fft, X_train, y_train, cv=10)
scores.mean()

0.9783327472699213

In [258]:
rf_fft = rf_fft.fit(X_train, y_train)
y_pred = rf_fft.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9762764311500773

In [259]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9471    0.9858    0.9661       563
      stairs     0.7333    0.4151    0.5301        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9201    0.8502    0.8740      1939
weighted avg     0.9774    0.9799    0.9773      1939



# Random Forest on Stat + FFT Data

In [260]:
rf = RandomForestClassifier(n_estimators=50, max_depth=None)

scores = cross_val_score(rf, X_train, y_train, cv=10)
scores.mean()

0.9778917345523453

In [261]:
rf = rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9767921609076844

In [262]:
print(classification_report(y_test,
                            y_test_proba,
                            digits=4,
                            target_names=activities))

              precision    recall  f1-score   support

        idle     1.0000    1.0000    1.0000       310
     running     0.9471    0.9858    0.9661       563
      stairs     0.7333    0.4151    0.5301        53
     walking     1.0000    1.0000    1.0000      1013

    accuracy                         0.9799      1939
   macro avg     0.9201    0.8502    0.8740      1939
weighted avg     0.9774    0.9799    0.9773      1939



In [263]:
rf.feature_importances_

array([0.12135068, 0.0030346 , 0.00285464, 0.00189093, 0.00207981,
       0.00264705, 0.00267364, 0.0017768 , 0.00228801, 0.00183725,
       0.00215745, 0.00130575, 0.00116453, 0.00142033, 0.00125875,
       0.00240186, 0.0023688 , 0.00178798, 0.00116315, 0.00192884,
       0.00143   , 0.00256934, 0.00185108, 0.00137953, 0.00213743,
       0.0017827 , 0.00145279, 0.00145299, 0.00187163, 0.00355518,
       0.25831801, 0.00247096, 0.0037207 , 0.00140746, 0.00174279,
       0.0010125 , 0.00207364, 0.00100137, 0.0011015 , 0.00766181,
       0.00766709, 0.00445099, 0.00494084, 0.00185406, 0.00111291,
       0.00897417, 0.00599008, 0.02360996, 0.02002389, 0.00834012,
       0.01225433, 0.02311026, 0.02692801, 0.00196822, 0.00099025,
       0.00155732, 0.00112686, 0.00123003, 0.0011921 , 0.00279292,
       0.16365602, 0.0017595 , 0.00168641, 0.0049187 , 0.0016013 ,
       0.00518947, 0.00329068, 0.00881295, 0.00405742, 0.00487349,
       0.00653459, 0.00712386, 0.00342154, 0.00592337, 0.00346

# Висновок

За результатами проведеного дослідження можна зробити висновок:

Показники Accuracy (0,98) свідчать, що моделі добре збалансовані та не перенавчені на тренувальних даних.

Точність (Precision) передбачення моделі на високому рівні. Для idle, walking становить 100%, для running 95%, stairs - 73%.

Повнота моделі (Recall): Кращі результати має багатокласова класифікація з використанням стратегії one-vs-one. Метод liner kernel має найгірші результати (менше 40%).
Найкращу повноту екземплярів класу виявлено в моделі швидкого перетворення Фур'є (49%) та значення гамма в Гуасівському ядрі (50,9%).

Всі моделі показали гарні результати класифікації класів для idle (100%), running (96%), walking (100%) на що вказують показники гармонічної середної точності і повноти. Але для класу stairs RandomRorest - 53%, liner kernel - 51% та scale gamma - 59%.

**В середньому для класифікації активності людини за даними акселерометра з мобільного телефону RandomRorest дозволяє простіше отримати прийнятні результати на часових фічах. Проте для класу stairs кращі показники показує використання швидкого перетворення Фур'є з використанням kernel='rbf' в стратегії one-vs-one - показник в 50,9%.**

# Варіант 2 _test in developing

## Create DataSet - option2

In [ ]:
def read_all_files(data_path):
    df2 = pd.DataFrame()
    read_file_count = 0
    dir_count = 0

    for root, _, files in os.walk(data_path):
        read_file_count += len(files)
        dir_count += 1

        for file in files:
            file_path = os.path.join(root, file)
            df2 = df2.append(pd.read_csv(file_path), ignore_index=True)

    return df2, read_file_count, dir_count

data_path = "/content/drive/MyDrive/ColabNotebooks/data/"
df2, read_file_count, dir_count = read_all_files(data_path)




In [86]:
directory_path = '/content/drive/MyDrive/ColabNotebooks/data/'

if os.path.exists(directory_path) and os.path.isdir(directory_path):
    print(f"Каталог {directory_path} існує.")
    contents = os.listdir(directory_path)
    print(f"Зміст: {contents}")
else:
    print(f"Каталог {directory_path} не існує.")


Каталог /content/drive/MyDrive/ColabNotebooks/data/ існує.
Зміст: ['idle', 'running', 'stairs', 'walking']


## Розподіл даних на навчальний та тестовий набори

In [87]:
df2.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,0.363919,0.023942,9.787497
1,0.392649,-0.028730,9.725247
2,0.426168,-0.043096,9.768343
3,0.426168,0.004788,9.739613
4,0.378284,-0.014365,9.744401


In [88]:
print(df2.columns)

Index(['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z'], dtype='object')


In [106]:
X_train, X_test, y_train, y_test = train_test_split(
    df2.drop(['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z'], axis=1),
    df2[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']],
    test_size=0.3,
    train_size=0.5,
    random_state=42
)

In [107]:
y_train_category = y_train.apply(lambda x: pd.cut(x, bins=[-float('inf'), 0, float('inf')], labels=['idle', 'running']))
y_test_category = y_test.apply(lambda x: pd.cut(x, bins=[-float('inf'), 0, float('inf')], labels=['idle', 'running']))

y_train_category = y_train_category.values
y_test_category = y_test_category.values


## Розрахунок часових ознак


In [99]:

features = [
    "mean",
    "std",
    "max",
    "min",
    "median",
    "mean_change",
    "std_change",
    "max_change",
    "min_change",
    "median_change",
]


## Навчання моделей

## Оцінка моделей

In [114]:
report_svm = classification_report(y_train_selected, y_pred_svm_train, digits=4, labels=[0, 1, 2, 3], target_names=['idle', 'running', 'stairs', 'walking'])
print(report_svm)


              precision    recall  f1-score   support

        idle     0.9967    0.9964    0.9965     13470
     running     0.9965    0.9944    0.9954      6045
      stairs     1.0000    0.9924    0.9962      5103
     walking     0.9950    0.9969    0.9960      3577

   micro avg     0.9971    0.9953    0.9962     28195
   macro avg     0.9971    0.9950    0.9960     28195
weighted avg     0.9971    0.9953    0.9962     28195

